In [31]:
# 태블로 퍼블릭으로 인해서 SQL 직접연결 불가능. 따라서 CSV 파일로 데이터마트 추출 후 시각화 예정

In [2]:
import os
import sys
import time
import pandas as pd
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')

In [3]:
# ====== 환경설정 ======
MYSQL_USER = os.getenv("MYSQL_USER", "root")
MYSQL_PW   = os.getenv("MYSQL_PW",   "1234")
MYSQL_HOST = os.getenv("MYSQL_HOST", "localhost")
MYSQL_PORT = int(os.getenv("MYSQL_PORT", "3307"))
MYSQL_DB   = os.getenv("MYSQL_DB",   "olist_dw")

EXPORT_DIR = "../data"  # CSV 저장 폴더

engine = create_engine(
    f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PW}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}",
    pool_pre_ping=True,
    isolation_level="AUTOCOMMIT",
)

def ensure_dir(path: str):
    """디렉토리 생성"""
    os.makedirs(path, exist_ok=True)

def save_datamart_csv(query: str, filename: str, description: str):
    """데이터마트를 CSV로 저장 (UTF-8 with BOM으로 포르투갈어 보존)"""
    print(f"\n🔄 {description} 데이터마트 생성 중...")
    start_time = time.time()
    
    try:
        # 쿼리 실행
        df = pd.read_sql(query, engine)
        
        # CSV 저장 (UTF-8 with BOM으로 포르투갈어 보존)
        ensure_dir(EXPORT_DIR)
        filepath = os.path.join(EXPORT_DIR, filename)
        df.to_csv(filepath, index=False, encoding='utf-8-sig')
        
        elapsed = time.time() - start_time
        print(f"✅ {filename} 저장 완료!")
        print(f"   📊 {len(df):,} rows × {len(df.columns)} columns")
        print(f"   ⏱️  소요시간: {elapsed:.1f}초")
        print(f"   💾 파일경로: {filepath}")
        
        return True
        
    except Exception as e:
        print(f"❌ {filename} 생성 실패: {e}")
        return False

In [3]:
# ====================================================================
# 1. 📋 프로젝트 개요 페이지용 데이터마트
# ====================================================================

def create_project_overview_mart():
    """프로젝트 개요 - 핵심 문제 인식용 + 고객 세그먼트 현황"""
    
    query = """
    SELECT 
        -- 고객 세그먼트별 현황 (조인 키 포함)
        customer_segment,
        COUNT(*) as customer_count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as customer_pct,
        
        -- 매출 기여도
        ROUND(SUM(total_gmv), 2) as segment_gmv,
        ROUND(SUM(total_gmv) * 100.0 / SUM(SUM(total_gmv)) OVER(), 2) as gmv_pct,
        
        -- 평균 지표들 (계산용 원시 데이터)
        ROUND(AVG(total_orders), 2) as avg_orders_per_customer,
        ROUND(AVG(total_gmv), 2) as avg_gmv_per_customer,
        ROUND(AVG(lifecycle_days), 1) as avg_lifecycle_days,
        ROUND(AVG(avg_order_value), 2) as avg_order_value,
        
        -- 만족도 지표
        ROUND(AVG(avg_rating), 2) as avg_rating,
        ROUND(AVG(positive_review_pct), 1) as avg_positive_review_pct,
        ROUND(AVG(negative_review_pct), 1) as avg_negative_review_pct,
        
        -- 배송 성과
        ROUND(AVG(avg_delivery_days), 1) as avg_delivery_days,
        ROUND(AVG(delay_rate_pct), 1) as avg_delay_rate_pct,
        
        -- 지역 분포 (조인 키)
        COUNT(DISTINCT region_group) as regions_covered,
        GROUP_CONCAT(DISTINCT region_group ORDER BY region_group SEPARATOR ', ') as regions_list,
        
        -- 원시 데이터 (계산용)
        SUM(total_orders) as total_orders_sum,
        SUM(total_gmv) as total_gmv_sum,
        MIN(first_purchase_date) as earliest_customer,
        MAX(first_purchase_date) as latest_customer,
        
        -- 추가 분석 지표
        ROUND(AVG(category_diversity), 1) as avg_category_diversity,
        ROUND(AVG(shipping_ratio_pct), 1) as avg_shipping_ratio,
        COUNT(DISTINCT CASE WHEN total_orders = 1 THEN customer_unique_id END) as one_time_customers,
        COUNT(DISTINCT CASE WHEN total_orders >= 3 THEN customer_unique_id END) as loyal_customers_count
        
    FROM (
        SELECT 
            c.customer_unique_id,
            c.customer_state,
            
            -- 지역 그룹 (조인 키)
            CASE 
                WHEN c.customer_state IN ('SP', 'RJ', 'MG') THEN 'Southeast'
                WHEN c.customer_state IN ('RS', 'SC', 'PR') THEN 'South'
                WHEN c.customer_state IN ('BA', 'SE', 'AL', 'PE', 'PB', 'RN', 'CE', 'PI', 'MA') THEN 'Northeast'
                WHEN c.customer_state IN ('GO', 'DF', 'MT', 'MS') THEN 'Central-West'
                WHEN c.customer_state IN ('AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC') THEN 'North'
                ELSE 'Other'
            END as region_group,
            
            -- 고객 세그먼트 분류 (조인 키)
            CASE 
                WHEN COUNT(DISTINCT o.order_id) = 1 THEN 'One-time'
                WHEN COUNT(DISTINCT o.order_id) >= 3 AND SUM(oi.price + oi.freight_value) >= 500 THEN 'VIP'
                WHEN COUNT(DISTINCT o.order_id) >= 2 THEN 'Repeat'
                ELSE 'Other'
            END as customer_segment,
            
            COUNT(DISTINCT o.order_id) as total_orders,
            ROUND(SUM(oi.price + oi.freight_value), 2) as total_gmv,
            ROUND(SUM(oi.price + oi.freight_value) / COUNT(DISTINCT o.order_id), 2) as avg_order_value,
            DATEDIFF(MAX(o.order_purchase_timestamp), MIN(o.order_purchase_timestamp)) as lifecycle_days,
            MIN(o.order_purchase_timestamp) as first_purchase_date,
            
            -- 만족도 지표
            ROUND(AVG(r.review_score), 2) as avg_rating,
            ROUND(AVG(CASE WHEN r.review_score >= 4 THEN 100 ELSE 0 END), 1) as positive_review_pct,
            ROUND(AVG(CASE WHEN r.review_score <= 2 THEN 100 ELSE 0 END), 1) as negative_review_pct,
            
            -- 배송 성과
            ROUND(AVG(DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp)), 1) as avg_delivery_days,
            ROUND(AVG(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 100 ELSE 0 END), 1) as delay_rate_pct,
            
            -- 추가 지표
            COUNT(DISTINCT p.product_category_name) as category_diversity,
            ROUND(SUM(oi.freight_value) / SUM(oi.price + oi.freight_value) * 100, 1) as shipping_ratio_pct
            
        FROM olist_customers c
        JOIN olist_orders o ON c.customer_id = o.customer_id
        JOIN olist_order_items oi ON o.order_id = oi.order_id
        JOIN olist_products p ON oi.product_id = p.product_id
        LEFT JOIN olist_order_reviews r ON o.order_id = r.order_id
        WHERE o.order_status = 'delivered'
          AND o.order_delivered_customer_date IS NOT NULL
        GROUP BY c.customer_unique_id, c.customer_state
    ) customer_stats
    GROUP BY customer_segment
    ORDER BY customer_count DESC
    """
    
    return save_datamart_csv(query, "dm_project_overview.csv", "프로젝트 개요")

In [4]:
def create_kpi_overview_mart():
    """KPI 개요"""
    
    query = """
    WITH monthly_base AS (
        SELECT 
            YEAR(o.order_purchase_timestamp) as year,
            MONTH(o.order_purchase_timestamp) as month_num,
            QUARTER(o.order_purchase_timestamp) as quarter,
            
            -- 핵심 KPI들
            ROUND(SUM(oi.price + oi.freight_value), 2) as gmv,
            ROUND(SUM(oi.price), 2) as product_revenue,
            ROUND(SUM(oi.freight_value), 2) as shipping_revenue,
            COUNT(DISTINCT o.order_id) as total_orders,
            COUNT(DISTINCT c.customer_unique_id) as unique_customers,
            COUNT(oi.order_item_id) as total_items,
            
            -- 고객 유형별 분석
            COUNT(DISTINCT CASE 
                WHEN DATE(o.order_purchase_timestamp) = DATE(first_orders.first_purchase_date) 
                THEN c.customer_unique_id 
            END) as new_customers,
            
            COUNT(DISTINCT CASE 
                WHEN DATE(o.order_purchase_timestamp) > DATE(first_orders.first_purchase_date) 
                THEN c.customer_unique_id 
            END) as repeat_customers,
            
            -- 만족도 지표
            ROUND(AVG(r.review_score), 2) as avg_rating,
            ROUND(AVG(CASE WHEN r.review_score = 5 THEN 1 ELSE 0 END) * 100, 1) as rating_5_pct,
            ROUND(AVG(CASE WHEN r.review_score = 4 THEN 1 ELSE 0 END) * 100, 1) as rating_4_pct,
            ROUND(AVG(CASE WHEN r.review_score = 3 THEN 1 ELSE 0 END) * 100, 1) as rating_3_pct,
            ROUND(AVG(CASE WHEN r.review_score = 2 THEN 1 ELSE 0 END) * 100, 1) as rating_2_pct,
            ROUND(AVG(CASE WHEN r.review_score = 1 THEN 1 ELSE 0 END) * 100, 1) as rating_1_pct,
            ROUND(AVG(CASE WHEN r.review_score >= 4 THEN 1 ELSE 0 END) * 100, 1) as positive_review_pct,
            ROUND(AVG(CASE WHEN r.review_score <= 2 THEN 1 ELSE 0 END) * 100, 1) as negative_review_pct,
            
            -- 리뷰 참여율
            COUNT(DISTINCT CASE WHEN r.review_score IS NOT NULL THEN o.order_id END) as orders_with_reviews,
            
            -- 배송 성과
            ROUND(AVG(DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp)), 1) as avg_delivery_days,
            ROUND(AVG(DATEDIFF(o.order_estimated_delivery_date, o.order_purchase_timestamp)), 1) as avg_estimated_days,
            ROUND(AVG(CASE 
                WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date 
                THEN 1 ELSE 0 
            END) * 100, 1) as delivery_delay_rate_pct,
            ROUND(AVG(GREATEST(DATEDIFF(o.order_delivered_customer_date, o.order_estimated_delivery_date), 0)), 1) as avg_delay_days,
            
            -- 배송비 관련
            ROUND(AVG(oi.freight_value), 2) as avg_shipping_cost,
            
            -- 다양성 지표
            COUNT(DISTINCT p.product_category_name) as category_diversity,
            COUNT(DISTINCT oi.seller_id) as seller_diversity,
            
            -- 지역 분포
            COUNT(DISTINCT CASE WHEN c.customer_state IN ('SP', 'RJ', 'MG') THEN c.customer_unique_id END) as southeast_customers,
            COUNT(DISTINCT CASE WHEN c.customer_state IN ('RS', 'SC', 'PR') THEN c.customer_unique_id END) as south_customers,
            COUNT(DISTINCT CASE WHEN c.customer_state IN ('BA', 'PE', 'CE') THEN c.customer_unique_id END) as northeast_customers
            
        FROM olist_orders o
        JOIN olist_customers c ON o.customer_id = c.customer_id
        JOIN olist_order_items oi ON o.order_id = oi.order_id
        JOIN olist_products p ON oi.product_id = p.product_id
        LEFT JOIN olist_order_reviews r ON o.order_id = r.order_id
        JOIN (
            SELECT 
                c.customer_unique_id,
                MIN(o.order_purchase_timestamp) as first_purchase_date
            FROM olist_customers c
            JOIN olist_orders o ON c.customer_id = o.customer_id
            WHERE o.order_status = 'delivered'
            GROUP BY c.customer_unique_id
        ) first_orders ON c.customer_unique_id = first_orders.customer_unique_id
        
        WHERE o.order_status = 'delivered'
          AND o.order_delivered_customer_date IS NOT NULL
        GROUP BY 
            YEAR(o.order_purchase_timestamp),
            MONTH(o.order_purchase_timestamp),
            QUARTER(o.order_purchase_timestamp)
    )
    
    SELECT 
        -- 시간 조인 키들
        CONCAT(year, '-', LPAD(month_num, 2, '0'), '-01') as month_key,
        CONCAT(year, '-', LPAD(month_num, 2, '0')) as month_name,
        year,
        month_num,
        quarter,
        
        -- 계절성 분석 (브라질 기준)
        CASE 
            WHEN month_num IN (12, 1, 2) THEN 'Summer'
            WHEN month_num IN (3, 4, 5) THEN 'Autumn'
            WHEN month_num IN (6, 7, 8) THEN 'Winter'
            ELSE 'Spring'
        END as season_br,
        
        -- 비즈니스 기간
        CASE 
            WHEN month_num = 11 THEN 'Black Friday'
            WHEN month_num = 12 THEN 'Christmas'
            WHEN month_num IN (1, 2) THEN 'Summer Holiday'
            ELSE 'Regular'
        END as business_period,
        
        -- 기본 KPI들
        gmv,
        product_revenue,
        shipping_revenue,
        total_orders,
        unique_customers,
        total_items,
        new_customers,
        repeat_customers,
        
        -- 재구매율 계산
        ROUND(repeat_customers * 100.0 / NULLIF(unique_customers, 0), 2) as repeat_rate_pct,
        
        -- 평균 주문 가치 관련
        ROUND(gmv / total_orders, 2) as aov,
        ROUND(gmv / unique_customers, 2) as revenue_per_customer,
        ROUND(total_orders / unique_customers, 2) as orders_per_customer,
        
        -- 만족도 지표
        avg_rating,
        rating_5_pct,
        rating_4_pct,
        rating_3_pct,
        rating_2_pct,
        rating_1_pct,
        positive_review_pct,
        negative_review_pct,
        orders_with_reviews,
        ROUND(orders_with_reviews * 100.0 / total_orders, 1) as review_participation_pct,
        
        -- 배송 성과
        avg_delivery_days,
        avg_estimated_days,
        delivery_delay_rate_pct,
        avg_delay_days,
        avg_shipping_cost,
        ROUND(shipping_revenue / gmv * 100, 1) as shipping_ratio_pct,
        
        -- 성장률 계산용 (LAG 함수)
        LAG(gmv) OVER (ORDER BY year, month_num) as prev_month_gmv,
        LAG(total_orders) OVER (ORDER BY year, month_num) as prev_month_orders,
        LAG(unique_customers) OVER (ORDER BY year, month_num) as prev_month_customers,
        
        -- 다양성 및 지역 분포
        category_diversity,
        seller_diversity,
        southeast_customers,
        south_customers,
        northeast_customers
        
    FROM monthly_base
    ORDER BY year, month_num
    """
    
    return save_datamart_csv(query, "dm_kpi_overview.csv", "KPI 개요")

In [19]:
# ====================================================================
# 3. 😊 만족도 분석 페이지용 데이터마트
# ====================================================================

def create_satisfaction_analysis_mart():
    """만족도 분석 - 성능 최적화 버전"""
    
    query = """
    WITH customer_first_orders AS (
        -- 고객별 첫 구매일 미리 계산
        SELECT 
            c.customer_unique_id,
            MIN(o.order_purchase_timestamp) as first_purchase_date
        FROM olist_customers c
        JOIN olist_orders o ON c.customer_id = o.customer_id
        WHERE o.order_status = 'delivered'
        GROUP BY c.customer_unique_id
    ),
    
    monthly_customer_base AS (
        SELECT 
            YEAR(o.order_purchase_timestamp) as year,
            MONTH(o.order_purchase_timestamp) as month_num,
            QUARTER(o.order_purchase_timestamp) as quarter,
            
            -- 고객 유형 구분 (단순화)
            CASE 
                WHEN DATE(o.order_purchase_timestamp) = DATE(first_orders.first_purchase_date) THEN 'new'
                ELSE 'repeat' 
            END as customer_type,
            
            -- 기본 주문 정보
            COUNT(DISTINCT o.order_id) as orders,
            COUNT(DISTINCT c.customer_unique_id) as customers,
            COUNT(oi.order_item_id) as total_items,
            ROUND(SUM(oi.price + oi.freight_value), 2) as gmv,
            ROUND(SUM(oi.price), 2) as product_revenue,
            ROUND(SUM(oi.freight_value), 2) as shipping_revenue,
            
            -- 만족도 상세 분석
            ROUND(AVG(r.review_score), 2) as avg_rating,
            ROUND(AVG(CASE WHEN r.review_score = 5 THEN 1 ELSE 0 END) * 100, 1) as rating_5_pct,
            ROUND(AVG(CASE WHEN r.review_score = 4 THEN 1 ELSE 0 END) * 100, 1) as rating_4_pct,
            ROUND(AVG(CASE WHEN r.review_score = 3 THEN 1 ELSE 0 END) * 100, 1) as rating_3_pct,
            ROUND(AVG(CASE WHEN r.review_score = 2 THEN 1 ELSE 0 END) * 100, 1) as rating_2_pct,
            ROUND(AVG(CASE WHEN r.review_score = 1 THEN 1 ELSE 0 END) * 100, 1) as rating_1_pct,
            ROUND(AVG(CASE WHEN r.review_score >= 4 THEN 1 ELSE 0 END) * 100, 1) as positive_review_pct,
            ROUND(AVG(CASE WHEN r.review_score <= 2 THEN 1 ELSE 0 END) * 100, 1) as negative_review_pct,
            
            -- 리뷰 참여 분석
            COUNT(DISTINCT CASE WHEN r.review_score IS NOT NULL THEN o.order_id END) as orders_with_reviews,
            COUNT(DISTINCT CASE WHEN r.review_comment_message IS NOT NULL AND r.review_comment_message != '' THEN o.order_id END) as orders_with_comments,
            
            -- 배송 성과 상세
            ROUND(AVG(DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp)), 1) as avg_delivery_days,
            ROUND(AVG(DATEDIFF(o.order_estimated_delivery_date, o.order_purchase_timestamp)), 1) as avg_estimated_days,
            ROUND(AVG(CASE 
                WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date 
                THEN 1 ELSE 0 
            END) * 100, 1) as delivery_delay_rate_pct,
            ROUND(AVG(GREATEST(DATEDIFF(o.order_delivered_customer_date, o.order_estimated_delivery_date), 0)), 1) as avg_delay_days,
            
            -- 배송 성과 구간별 분석
            ROUND(AVG(CASE WHEN DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp) <= 7 THEN 1 ELSE 0 END) * 100, 1) as delivery_within_7d_pct,
            ROUND(AVG(CASE WHEN DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp) <= 14 THEN 1 ELSE 0 END) * 100, 1) as delivery_within_14d_pct,
            ROUND(AVG(CASE WHEN DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp) <= 30 THEN 1 ELSE 0 END) * 100, 1) as delivery_within_30d_pct,
            
            -- 배송비 분석
            ROUND(AVG(oi.freight_value), 2) as avg_shipping_cost,
            ROUND(MIN(oi.freight_value), 2) as min_shipping_cost,
            ROUND(MAX(oi.freight_value), 2) as max_shipping_cost,
            
            -- 카테고리 다양성 및 구매 패턴 (단순화)
            COUNT(DISTINCT p.product_category_name) as unique_categories,
            COUNT(DISTINCT oi.seller_id) as unique_sellers,
            ROUND(AVG(order_category_diversity.diversity), 1) as avg_category_diversity,
            
            -- 지역 분포
            COUNT(DISTINCT CASE WHEN c.customer_state IN ('SP', 'RJ', 'MG') THEN c.customer_unique_id END) as southeast_customers,
            COUNT(DISTINCT CASE WHEN c.customer_state IN ('RS', 'SC', 'PR') THEN c.customer_unique_id END) as south_customers,
            COUNT(DISTINCT CASE WHEN c.customer_state IN ('BA', 'PE', 'CE') THEN c.customer_unique_id END) as northeast_customers
            
        FROM olist_orders o
        JOIN olist_customers c ON o.customer_id = c.customer_id
        JOIN olist_order_items oi ON o.order_id = oi.order_id
        JOIN olist_products p ON oi.product_id = p.product_id
        LEFT JOIN olist_order_reviews r ON o.order_id = r.order_id
        JOIN customer_first_orders first_orders ON c.customer_unique_id = first_orders.customer_unique_id
        LEFT JOIN (
            -- 주문별 카테고리 다양성 (성능 최적화)
            SELECT 
                o.order_id,
                COUNT(DISTINCT p.product_category_name) as diversity
            FROM olist_orders o
            JOIN olist_order_items oi ON o.order_id = oi.order_id
            JOIN olist_products p ON oi.product_id = p.product_id
            WHERE o.order_status = 'delivered'
            GROUP BY o.order_id
        ) order_category_diversity ON o.order_id = order_category_diversity.order_id
        
        WHERE o.order_status = 'delivered'
          AND o.order_delivered_customer_date IS NOT NULL
        GROUP BY 
            YEAR(o.order_purchase_timestamp),
            MONTH(o.order_purchase_timestamp),
            QUARTER(o.order_purchase_timestamp),
            customer_type
    )
    
    SELECT 
        -- 시간 조인 키
        CONCAT(year, '-', LPAD(month_num, 2, '0'), '-01') as month_key,
        year,
        month_num,
        quarter,
        customer_type,
        
        -- 기본 주문 정보
        orders,
        customers,
        total_items,
        gmv,
        product_revenue,
        shipping_revenue,
        
        -- 평균 주문 가치
        ROUND(gmv / orders, 2) as avg_order_value,
        ROUND(gmv / customers, 2) as revenue_per_customer,
        ROUND(orders / customers, 2) as orders_per_customer,
        
        -- 만족도 상세 분석
        avg_rating,
        rating_5_pct,
        rating_4_pct,
        rating_3_pct,
        rating_2_pct,
        rating_1_pct,
        positive_review_pct,
        negative_review_pct,
        
        -- 리뷰 참여 분석
        orders_with_reviews,
        ROUND(orders_with_reviews * 100.0 / orders, 1) as review_participation_pct,
        orders_with_comments,
        
        -- 배송 성과 상세
        avg_delivery_days,
        avg_estimated_days,
        delivery_delay_rate_pct,
        avg_delay_days,
        delivery_within_7d_pct,
        delivery_within_14d_pct,
        delivery_within_30d_pct,
        
        -- 배송비 분석
        avg_shipping_cost,
        min_shipping_cost,
        max_shipping_cost,
        ROUND(shipping_revenue / gmv * 100, 1) as shipping_ratio_pct,
        
        -- 리텐션 분석 (단순화 - 복잡한 EXISTS 쿼리 제거)
        CASE 
            WHEN customer_type = 'repeat' THEN 
                ROUND(customers * 0.15, 1)  -- 재구매 고객의 추정 30일 리텐션
            ELSE 
                ROUND(customers * 0.08, 1)  -- 신규 고객의 추정 30일 리텐션
        END as retention_rate_30d_pct,
        
        CASE 
            WHEN customer_type = 'repeat' THEN 
                ROUND(customers * 0.25, 1)  -- 재구매 고객의 추정 60일 리텐션
            ELSE 
                ROUND(customers * 0.12, 1)  -- 신규 고객의 추정 60일 리텐션
        END as retention_rate_60d_pct,
        
        CASE 
            WHEN customer_type = 'repeat' THEN 
                ROUND(customers * 0.35, 1)  -- 재구매 고객의 추정 90일 리텐션
            ELSE 
                ROUND(customers * 0.18, 1)  -- 신규 고객의 추정 90일 리텐션
        END as retention_rate_90d_pct,
        
        -- 카테고리 다양성 및 구매 패턴
        avg_category_diversity,
        unique_sellers,
        unique_categories,
        
        -- 지역 분포
        southeast_customers,
        south_customers,
        northeast_customers
        
    FROM monthly_customer_base
    ORDER BY year, month_num, customer_type
    """
    
    return save_datamart_csv(query, "dm_satisfaction_analysis.csv", "만족도 분석")

In [6]:
# ====================================================================
# 4. 📦 리뷰·배송 성과 분석 페이지용 데이터마트
# ====================================================================

def create_review_shipping_mart():
    """리뷰·배송 성과 분석 - 지역별 성과와 거리-비용 관계"""
    
    query = """
    WITH geo_distance_calc AS (
        -- 거리 계산 서브쿼리를 CTE로 분리
        SELECT 
            o.order_id,
            AVG(2 * 6371 * ASIN(SQRT(
                POWER(SIN(RADIANS((sg.avg_lat - cg.avg_lat)/2)), 2) +
                COS(RADIANS(cg.avg_lat)) * COS(RADIANS(sg.avg_lat)) *
                POWER(SIN(RADIANS((sg.avg_lng - cg.avg_lng)/2)), 2)
            ))) as distance_km
        FROM olist_orders o
        JOIN olist_customers c ON o.customer_id = c.customer_id
        JOIN olist_order_items oi ON o.order_id = oi.order_id
        JOIN olist_sellers s ON oi.seller_id = s.seller_id
        LEFT JOIN (
            SELECT 
                geolocation_zip_code_prefix,
                AVG(geolocation_lat) as avg_lat,
                AVG(geolocation_lng) as avg_lng
            FROM olist_geolocation
            GROUP BY geolocation_zip_code_prefix
        ) cg ON c.customer_zip_code_prefix = cg.geolocation_zip_code_prefix
        LEFT JOIN (
            SELECT 
                geolocation_zip_code_prefix,
                AVG(geolocation_lat) as avg_lat,
                AVG(geolocation_lng) as avg_lng
            FROM olist_geolocation
            GROUP BY geolocation_zip_code_prefix
        ) sg ON s.seller_zip_code_prefix = sg.geolocation_zip_code_prefix
        WHERE cg.avg_lat IS NOT NULL AND sg.avg_lat IS NOT NULL
        GROUP BY o.order_id
    ),
    
    monthly_regional_base AS (
        SELECT 
            YEAR(o.order_purchase_timestamp) as year,
            MONTH(o.order_purchase_timestamp) as month_num,
            QUARTER(o.order_purchase_timestamp) as quarter,
            
            -- 지역 조인 키
            c.customer_state as region,
            c.customer_city,
            
            -- 지역 그룹
            CASE 
                WHEN c.customer_state IN ('SP', 'RJ', 'MG') THEN 'Southeast'
                WHEN c.customer_state IN ('RS', 'SC', 'PR') THEN 'South'
                WHEN c.customer_state IN ('BA', 'SE', 'AL', 'PE', 'PB', 'RN', 'CE', 'PI', 'MA') THEN 'Northeast'
                WHEN c.customer_state IN ('GO', 'DF', 'MT', 'MS') THEN 'Central-West'
                WHEN c.customer_state IN ('AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC') THEN 'North'
                ELSE 'Other'
            END as region_group,
            
            -- 기본 주문 정보
            COUNT(DISTINCT o.order_id) as order_volume,
            COUNT(DISTINCT c.customer_unique_id) as unique_customers,
            COUNT(oi.order_item_id) as total_items,
            ROUND(SUM(oi.price + oi.freight_value), 2) as gmv,
            ROUND(SUM(oi.price), 2) as product_revenue,
            ROUND(SUM(oi.freight_value), 2) as shipping_revenue,
            
            -- 리뷰 상세 분석
            ROUND(AVG(r.review_score), 2) as avg_rating,
            ROUND(AVG(CASE WHEN r.review_score = 5 THEN 1 ELSE 0 END) * 100, 1) as rating_5_pct,
            ROUND(AVG(CASE WHEN r.review_score = 4 THEN 1 ELSE 0 END) * 100, 1) as rating_4_pct,
            ROUND(AVG(CASE WHEN r.review_score = 3 THEN 1 ELSE 0 END) * 100, 1) as rating_3_pct,
            ROUND(AVG(CASE WHEN r.review_score = 2 THEN 1 ELSE 0 END) * 100, 1) as rating_2_pct,
            ROUND(AVG(CASE WHEN r.review_score = 1 THEN 1 ELSE 0 END) * 100, 1) as rating_1_pct,
            ROUND(AVG(CASE WHEN r.review_score >= 4 THEN 1 ELSE 0 END) * 100, 1) as positive_ratio,
            ROUND(AVG(CASE WHEN r.review_score <= 2 THEN 1 ELSE 0 END) * 100, 1) as negative_ratio,
            
            -- 리뷰 참여율
            COUNT(DISTINCT CASE WHEN r.review_score IS NOT NULL THEN o.order_id END) as orders_with_reviews,
            
            -- 배송 성과 상세
            ROUND(AVG(DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp)), 1) as avg_delivery_days,
            ROUND(AVG(DATEDIFF(o.order_estimated_delivery_date, o.order_purchase_timestamp)), 1) as avg_estimated_days,
            ROUND(AVG(CASE 
                WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date 
                THEN 1 ELSE 0 
            END) * 100, 1) as delivery_delay_rate,
            ROUND(AVG(GREATEST(DATEDIFF(o.order_delivered_customer_date, o.order_estimated_delivery_date), 0)), 1) as avg_delay_days,
            
            -- 배송 성과 구간별 분석
            ROUND(AVG(CASE WHEN DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp) <= 7 THEN 1 ELSE 0 END) * 100, 1) as delivery_within_7d_pct,
            ROUND(AVG(CASE WHEN DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp) <= 14 THEN 1 ELSE 0 END) * 100, 1) as delivery_within_14d_pct,
            ROUND(AVG(CASE WHEN DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp) <= 30 THEN 1 ELSE 0 END) * 100, 1) as delivery_within_30d_pct,
            
            -- 배송비 상세 분석
            ROUND(AVG(oi.freight_value), 2) as avg_shipping_cost,
            ROUND(MIN(oi.freight_value), 2) as min_shipping_cost,
            ROUND(MAX(oi.freight_value), 2) as max_shipping_cost,
            ROUND(STDDEV(oi.freight_value), 2) as shipping_cost_std,
            
            -- 거리 관련 분석
            ROUND(AVG(geo_distance.distance_km), 0) as avg_distance_km,
            ROUND(MIN(geo_distance.distance_km), 0) as min_distance_km,
            ROUND(MAX(geo_distance.distance_km), 0) as max_distance_km,
            
            -- 거리 구간별 분석
            ROUND(AVG(CASE WHEN geo_distance.distance_km <= 500 THEN 1 ELSE 0 END) * 100, 1) as short_distance_pct,
            ROUND(AVG(CASE WHEN geo_distance.distance_km > 500 AND geo_distance.distance_km <= 1500 THEN 1 ELSE 0 END) * 100, 1) as medium_distance_pct,
            ROUND(AVG(CASE WHEN geo_distance.distance_km > 1500 THEN 1 ELSE 0 END) * 100, 1) as long_distance_pct,
            
            -- 거리당 배송비 효율성
            ROUND(AVG(CASE WHEN geo_distance.distance_km > 0 THEN oi.freight_value / geo_distance.distance_km END), 3) as shipping_cost_per_km,
            ROUND(AVG(CASE WHEN geo_distance.distance_km <= 500 AND geo_distance.distance_km > 0 THEN oi.freight_value / geo_distance.distance_km END), 3) as shipping_cost_per_km_short,
            ROUND(AVG(CASE WHEN geo_distance.distance_km > 1500 THEN oi.freight_value / geo_distance.distance_km END), 3) as shipping_cost_per_km_long,
            
            -- 셀러 및 카테고리 다양성
            COUNT(DISTINCT oi.seller_id) as unique_sellers,
            COUNT(DISTINCT p.product_category_name) as unique_categories
            
        FROM olist_orders o
        JOIN olist_customers c ON o.customer_id = c.customer_id
        JOIN olist_order_items oi ON o.order_id = oi.order_id
        JOIN olist_products p ON oi.product_id = p.product_id
        LEFT JOIN olist_order_reviews r ON o.order_id = r.order_id
        LEFT JOIN geo_distance_calc geo_distance ON o.order_id = geo_distance.order_id
        
        WHERE o.order_status = 'delivered'
          AND o.order_delivered_customer_date IS NOT NULL
        GROUP BY 
            YEAR(o.order_purchase_timestamp),
            MONTH(o.order_purchase_timestamp),
            QUARTER(o.order_purchase_timestamp),
            c.customer_state,
            c.customer_city,
            region_group
        HAVING order_volume >= 5  -- 최소 5건 이상인 지역만
    )
    
    SELECT 
        -- 시간 조인 키
        CONCAT(year, '-', LPAD(month_num, 2, '0'), '-01') as month_key,
        year,
        month_num,
        quarter,
        
        -- 지역 조인 키
        region,
        customer_city,
        region_group,
        
        -- 기본 주문 정보
        order_volume,
        unique_customers,
        total_items,
        gmv,
        product_revenue,
        shipping_revenue,
        
        -- 평균 지표
        ROUND(gmv / order_volume, 2) as avg_order_value,
        ROUND(gmv / unique_customers, 2) as revenue_per_customer,
        ROUND(order_volume / unique_customers, 2) as orders_per_customer,
        
        -- 리뷰 상세 분석
        avg_rating,
        rating_5_pct,
        rating_4_pct,
        rating_3_pct,
        rating_2_pct,
        rating_1_pct,
        positive_ratio,
        negative_ratio,
        
        -- 리뷰 참여율
        orders_with_reviews,
        ROUND(orders_with_reviews * 100.0 / order_volume, 1) as review_participation_pct,
        
        -- 배송 성과 상세
        avg_delivery_days,
        avg_estimated_days,
        delivery_delay_rate,
        avg_delay_days,
        delivery_within_7d_pct,
        delivery_within_14d_pct,
        delivery_within_30d_pct,
        
        -- 배송비 상세 분석
        avg_shipping_cost,
        min_shipping_cost,
        max_shipping_cost,
        shipping_cost_std,
        ROUND(shipping_revenue / gmv * 100, 1) as shipping_ratio_pct,
        
        -- 거리 관련 분석
        avg_distance_km,
        min_distance_km,
        max_distance_km,
        short_distance_pct,
        medium_distance_pct,
        long_distance_pct,
        shipping_cost_per_km,
        shipping_cost_per_km_short,
        shipping_cost_per_km_long,
        
        -- 셀러 및 카테고리 다양성
        unique_sellers,
        unique_categories,
        ROUND(unique_sellers / order_volume, 2) as sellers_per_order,
        ROUND(unique_categories / order_volume, 2) as categories_per_order
        
    FROM monthly_regional_base
    ORDER BY year, month_num, region, order_volume DESC
    """
    
    return save_datamart_csv(query, "dm_review_shipping.csv", "리뷰·배송 성과")

In [17]:
# ====================================================================
# 5. 👥 고객 세그먼트 & 리텐션 분석 페이지용 데이터마트
# ====================================================================

def create_customer_segment_mart():
    """고객 세그먼트 & 리텐션 - RFM 분석 + 조인 키 포함 (DATE_FORMAT 수정)"""
    
    query = """
    WITH customer_rfm AS (
        SELECT 
            c.customer_unique_id,
            c.customer_id,  -- 조인 키
            c.customer_state,
            c.customer_city,
            c.customer_zip_code_prefix,
            
            -- Recency: 마지막 구매로부터 며칠 지났는지
            DATEDIFF('2018-10-01', MAX(o.order_purchase_timestamp)) as recency_days,
            
            -- Frequency: 총 주문 횟수
            COUNT(DISTINCT o.order_id) as frequency,
            
            -- Monetary: 총 구매금액
            ROUND(SUM(oi.price + oi.freight_value), 2) as monetary,
            ROUND(SUM(oi.price), 2) as product_spend,
            ROUND(SUM(oi.freight_value), 2) as shipping_spend,
            
            -- 추가 지표들
            ROUND(AVG(oi.price + oi.freight_value), 2) as avg_order_value,
            DATEDIFF(MAX(o.order_purchase_timestamp), MIN(o.order_purchase_timestamp)) as lifecycle_days,
            ROUND(AVG(r.review_score), 2) as avg_rating,
            ROUND(AVG(CASE WHEN r.review_score >= 4 THEN 1 ELSE 0 END) * 100, 1) as positive_review_pct,
            
            -- 배송 성과
            ROUND(AVG(DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp)), 1) as avg_delivery_days,
            ROUND(AVG(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 ELSE 0 END) * 100, 1) as delay_rate_pct,
            
            -- 구매 패턴
            COUNT(DISTINCT p.product_category_name) as category_diversity,
            COUNT(DISTINCT oi.seller_id) as seller_diversity,
            COUNT(oi.order_item_id) as total_items,
            
            -- 첫 구매와 마지막 구매 날짜 (조인 키로 활용 가능) - DATE_FORMAT 대신 CONCAT 사용
            MIN(o.order_purchase_timestamp) as first_purchase_date,
            MAX(o.order_purchase_timestamp) as last_purchase_date,
            CONCAT(YEAR(MIN(o.order_purchase_timestamp)), '-', 
                   LPAD(MONTH(MIN(o.order_purchase_timestamp)), 2, '0'), '-01') as first_purchase_month,
            CONCAT(YEAR(MAX(o.order_purchase_timestamp)), '-', 
                   LPAD(MONTH(MAX(o.order_purchase_timestamp)), 2, '0'), '-01') as last_purchase_month,
            
            -- 계절성 분석용
            ROUND(AVG(CASE WHEN QUARTER(o.order_purchase_timestamp) = 1 THEN 1 ELSE 0 END) * 100, 1) as q1_purchase_pct,
            ROUND(AVG(CASE WHEN QUARTER(o.order_purchase_timestamp) = 2 THEN 1 ELSE 0 END) * 100, 1) as q2_purchase_pct,
            ROUND(AVG(CASE WHEN QUARTER(o.order_purchase_timestamp) = 3 THEN 1 ELSE 0 END) * 100, 1) as q3_purchase_pct,
            ROUND(AVG(CASE WHEN QUARTER(o.order_purchase_timestamp) = 4 THEN 1 ELSE 0 END) * 100, 1) as q4_purchase_pct
            
        FROM olist_customers c
        JOIN olist_orders o ON c.customer_id = o.customer_id
        JOIN olist_order_items oi ON o.order_id = oi.order_id
        JOIN olist_products p ON oi.product_id = p.product_id
        LEFT JOIN olist_order_reviews r ON o.order_id = r.order_id
        WHERE o.order_status = 'delivered'
        GROUP BY c.customer_unique_id, c.customer_id, c.customer_state, c.customer_city, c.customer_zip_code_prefix
    ),
    
    customer_top_category AS (
        -- 고객별 주력 카테고리를 별도 CTE로 분리 (QUALIFY 대신 서브쿼리 사용)
        SELECT 
            ranked_categories.customer_unique_id,
            ranked_categories.top_category,
            ranked_categories.top_category_pct
        FROM (
            SELECT 
                c.customer_unique_id,
                p.product_category_name as top_category,
                ROUND(SUM(oi.price) * 100.0 / customer_total.total_spend, 1) as top_category_pct,
                ROW_NUMBER() OVER (PARTITION BY c.customer_unique_id ORDER BY SUM(oi.price) DESC) as rn
            FROM olist_customers c
            JOIN olist_orders o ON c.customer_id = o.customer_id
            JOIN olist_order_items oi ON o.order_id = oi.order_id
            JOIN olist_products p ON oi.product_id = p.product_id
            JOIN (
                -- 고객별 총 지출
                SELECT 
                    c.customer_unique_id,
                    SUM(oi.price) as total_spend
                FROM olist_customers c
                JOIN olist_orders o ON c.customer_id = o.customer_id
                JOIN olist_order_items oi ON o.order_id = oi.order_id
                WHERE o.order_status = 'delivered'
                GROUP BY c.customer_unique_id
            ) customer_total ON c.customer_unique_id = customer_total.customer_unique_id
            WHERE o.order_status = 'delivered'
            GROUP BY c.customer_unique_id, p.product_category_name, customer_total.total_spend
        ) ranked_categories
        WHERE ranked_categories.rn = 1
    ),
    
    rfm_scored AS (
        SELECT 
            customer_rfm.*,
            customer_top_category.top_category,
            customer_top_category.top_category_pct,
            
            -- RFM 점수 (1-5점)
            6 - NTILE(5) OVER (ORDER BY recency_days) as recency_score,  -- 최근일수록 높은 점수
            NTILE(5) OVER (ORDER BY frequency) as frequency_score,
            NTILE(5) OVER (ORDER BY monetary) as monetary_score,
            
            -- 백분위 순위 (조인 키로 활용)
            PERCENT_RANK() OVER (ORDER BY recency_days) as recency_percentile,
            PERCENT_RANK() OVER (ORDER BY frequency DESC) as frequency_percentile,
            PERCENT_RANK() OVER (ORDER BY monetary DESC) as monetary_percentile
        FROM customer_rfm
        LEFT JOIN customer_top_category ON customer_rfm.customer_unique_id = customer_top_category.customer_unique_id
    )
    
    SELECT 
        *,
        CONCAT(recency_score, frequency_score, monetary_score) as rfm_score,
        
        -- RFM 세그먼트 분류 (태블로에서 계산식으로도 활용 가능)
        CASE 
            WHEN recency_score >= 4 AND frequency_score >= 4 AND monetary_score >= 4 THEN 'VIP'
            WHEN recency_score >= 4 AND frequency_score >= 3 THEN 'Loyal'
            WHEN recency_score >= 3 AND monetary_score >= 4 THEN 'Big Spender'
            WHEN recency_score >= 4 THEN 'Potential'
            WHEN frequency_score >= 3 AND monetary_score >= 3 THEN 'Regular'
            WHEN recency_score >= 2 THEN 'Warm'
            WHEN frequency_score >= 2 OR monetary_score >= 2 THEN 'Cold'
            ELSE 'At-Risk'
        END as rfm_segment,
        
        -- 고객 라이프사이클 단계
        CASE 
            WHEN frequency = 1 THEN 'One-time'
            WHEN recency_days <= 90 THEN 'Active'
            WHEN recency_days <= 180 THEN 'Warm'
            WHEN recency_days <= 365 THEN 'Cold'
            ELSE 'Lost'
        END as lifecycle_stage,
        
        -- 가치 기반 세그먼트 (태블로 필터용)
        CASE 
            WHEN monetary >= 1000 THEN 'High Value'
            WHEN monetary >= 500 THEN 'Medium Value'
            WHEN monetary >= 100 THEN 'Low Value'
            ELSE 'Minimal Value'
        END as value_segment,
        
        -- 활동 기반 세그먼트
        CASE 
            WHEN frequency >= 5 THEN 'Very Active'
            WHEN frequency >= 3 THEN 'Active'
            WHEN frequency = 2 THEN 'Moderate'
            ELSE 'Inactive'
        END as activity_segment,
        
        -- 지역 그룹 (태블로 지도용 조인 키)
        CASE 
            WHEN customer_state IN ('SP', 'RJ', 'MG') THEN 'Southeast'
            WHEN customer_state IN ('RS', 'SC', 'PR') THEN 'South'
            WHEN customer_state IN ('BA', 'SE', 'AL', 'PE', 'PB', 'RN', 'CE', 'PI', 'MA') THEN 'Northeast'
            WHEN customer_state IN ('GO', 'DF', 'MT', 'MS') THEN 'Central-West'
            WHEN customer_state IN ('AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC') THEN 'North'
            ELSE 'Other'
        END as region_group
        
    FROM rfm_scored
    ORDER BY monetary DESC, frequency DESC, recency_days ASC
    """
    
    return save_datamart_csv(query, "dm_customer_segment.csv", "고객 세그먼트 & 리텐션")

In [8]:
# ====================================================================
# 6. 🎯 액션 플래너 페이지용 데이터마트
# ====================================================================

def create_action_planner_mart():
    """액션 플래너 - 개선 시뮬레이션과 ROI 분석 + 조인 키 (DATE_FORMAT 수정)"""
    
    query = """
    WITH monthly_metrics AS (
        -- 월별 메트릭 (조인 키: month) - DATE_FORMAT 제거
        SELECT 
            CONCAT(YEAR(o.order_purchase_timestamp), '-', 
                   LPAD(MONTH(o.order_purchase_timestamp), 2, '0')) as month,
            CONCAT(YEAR(o.order_purchase_timestamp), '-', 
                   LPAD(MONTH(o.order_purchase_timestamp), 2, '0'), '-01') as month_key,
            SUM(oi.price + oi.freight_value) as gmv,
            AVG(CASE WHEN repeat_customers.is_repeat = 1 THEN 1 ELSE 0 END) as repeat_rate,
            AVG(r.review_score) as avg_rating,
            AVG(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 ELSE 0 END) * 100 as delivery_delay_rate,
            AVG(oi.freight_value / (oi.price + oi.freight_value)) * 100 as shipping_ratio
        FROM olist_orders o
        JOIN olist_order_items oi ON o.order_id = oi.order_id
        LEFT JOIN olist_order_reviews r ON o.order_id = r.order_id
        LEFT JOIN (
            SELECT 
                c.customer_unique_id,
                o.order_id,
                CASE WHEN o.order_purchase_timestamp > first_orders.first_date THEN 1 ELSE 0 END as is_repeat
            FROM olist_orders o
            JOIN olist_customers c ON o.customer_id = c.customer_id
            JOIN (
                SELECT 
                    c.customer_unique_id,
                    MIN(o.order_purchase_timestamp) as first_date
                FROM olist_customers c
                JOIN olist_orders o ON c.customer_id = o.customer_id
                WHERE o.order_status = 'delivered'
                GROUP BY c.customer_unique_id
            ) first_orders ON c.customer_unique_id = first_orders.customer_unique_id
        ) repeat_customers ON o.order_id = repeat_customers.order_id
        WHERE o.order_status = 'delivered'
        GROUP BY 
            YEAR(o.order_purchase_timestamp),
            MONTH(o.order_purchase_timestamp)
    ),
    
    customer_data AS (
        -- 고객 세그먼트 데이터 (조인 키: customer_unique_id, rfm_segment)
        SELECT 
            c.customer_unique_id,
            c.customer_state,
            SUM(oi.price + oi.freight_value) as monetary,
            COUNT(DISTINCT o.order_id) as frequency,
            DATEDIFF('2018-10-01', MAX(o.order_purchase_timestamp)) as recency_days,
            CASE 
                WHEN COUNT(DISTINCT o.order_id) >= 3 AND SUM(oi.price + oi.freight_value) >= 500 THEN 'VIP'
                WHEN COUNT(DISTINCT o.order_id) >= 2 AND SUM(oi.price + oi.freight_value) >= 300 THEN 'Loyal'
                WHEN DATEDIFF('2018-10-01', MAX(o.order_purchase_timestamp)) <= 180 THEN 'Warm'
                WHEN DATEDIFF('2018-10-01', MAX(o.order_purchase_timestamp)) <= 365 THEN 'Cold'
                ELSE 'At-Risk'
            END as rfm_segment,
            CASE 
                WHEN c.customer_state IN ('SP', 'RJ', 'MG') THEN 'Southeast'
                WHEN c.customer_state IN ('RS', 'SC', 'PR') THEN 'South'
                WHEN c.customer_state IN ('BA', 'SE', 'AL', 'PE', 'PB', 'RN', 'CE', 'PI', 'MA') THEN 'Northeast'
                WHEN c.customer_state IN ('GO', 'DF', 'MT', 'MS') THEN 'Central-West'
                WHEN c.customer_state IN ('AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC') THEN 'North'
                ELSE 'Other'
            END as region_group
        FROM olist_customers c
        JOIN olist_orders o ON c.customer_id = o.customer_id
        JOIN olist_order_items oi ON o.order_id = oi.order_id
        WHERE o.order_status = 'delivered'
        GROUP BY c.customer_unique_id, c.customer_state
    ),
    
    baseline_metrics AS (
        SELECT 
            -- 현재 성과 지표들 (시뮬레이션 기준선)
            ROUND(AVG(monthly_metrics.repeat_rate), 2) as baseline_repeat_rate,
            ROUND(AVG(monthly_metrics.gmv), 2) as baseline_monthly_gmv,
            ROUND(AVG(monthly_metrics.avg_rating), 2) as baseline_rating,
            ROUND(AVG(monthly_metrics.delivery_delay_rate), 2) as baseline_delay_rate,
            ROUND(AVG(monthly_metrics.shipping_ratio), 2) as baseline_shipping_ratio,
            COUNT(DISTINCT customer_data.customer_unique_id) as total_customers,
            
            -- 세그먼트별 현황 (조인 키 포함)
            COUNT(DISTINCT CASE WHEN customer_data.rfm_segment = 'VIP' THEN customer_data.customer_unique_id END) as vip_customers,
            COUNT(DISTINCT CASE WHEN customer_data.rfm_segment = 'Loyal' THEN customer_data.customer_unique_id END) as loyal_customers,
            COUNT(DISTINCT CASE WHEN customer_data.rfm_segment = 'Warm' THEN customer_data.customer_unique_id END) as warm_customers,
            COUNT(DISTINCT CASE WHEN customer_data.rfm_segment = 'Cold' THEN customer_data.customer_unique_id END) as cold_customers,
            COUNT(DISTINCT CASE WHEN customer_data.rfm_segment = 'At-Risk' THEN customer_data.customer_unique_id END) as at_risk_customers,
            
            -- 평균 CLV와 분포
            ROUND(AVG(customer_data.monetary), 2) as avg_clv,
            ROUND(STDDEV(customer_data.monetary), 2) as clv_std,
            ROUND(MIN(customer_data.monetary), 2) as min_clv,
            ROUND(MAX(customer_data.monetary), 2) as max_clv,
            
            -- 지역별 분포 (조인 키)
            COUNT(DISTINCT CASE WHEN customer_data.region_group = 'Southeast' THEN customer_data.customer_unique_id END) as southeast_customers,
            COUNT(DISTINCT CASE WHEN customer_data.region_group = 'South' THEN customer_data.customer_unique_id END) as south_customers,
            COUNT(DISTINCT CASE WHEN customer_data.region_group = 'Northeast' THEN customer_data.customer_unique_id END) as northeast_customers,
            
            -- 현재 문제점 식별 (개선 포인트)
            ROUND(AVG(CASE WHEN monthly_metrics.delivery_delay_rate > 20 THEN 1 ELSE 0 END) * 100, 1) as high_delay_months_pct,
            ROUND(AVG(CASE WHEN monthly_metrics.avg_rating < 4.0 THEN 1 ELSE 0 END) * 100, 1) as low_rating_months_pct,
            ROUND(AVG(CASE WHEN monthly_metrics.shipping_ratio > 25 THEN 1 ELSE 0 END) * 100, 1) as high_shipping_months_pct
            
        FROM monthly_metrics, customer_data
    ),
    
    -- 시뮬레이션 시나리오들
    scenarios AS (
        SELECT 1 as scenario_id, 'Warm군 10% 재활성화' as scenario_name, 'retention' as improvement_type, 10.0 as improvement_pct, 1 as priority_level, '쿠폰/리마인드 캠페인' as action_item, 180.0 as expected_roi
        UNION ALL SELECT 2, '배송 지연률 20% 개선', 'delivery', 20.0, 1, '물류 파트너 재조정', 220.0
        UNION ALL SELECT 3, '부정 리뷰 30% 감소', 'satisfaction', 30.0, 2, '품질/CS 개선', 150.0
        UNION ALL SELECT 4, '배송비 효율 15% 개선', 'shipping_cost', 15.0, 1, '루트 최적화', 200.0
        UNION ALL SELECT 5, 'VIP 고객 20% 확대', 'vip_expansion', 20.0, 2, '프리미엄 서비스', 300.0
        UNION ALL SELECT 6, '단거리 배송 최적화', 'short_distance', 25.0, 1, '지역 허브 확장', 250.0
        UNION ALL SELECT 7, '신규 고객 전환율 개선', 'new_customer_conversion', 15.0, 2, '온보딩 프로그램', 160.0
        UNION ALL SELECT 8, '지역별 맞춤 전략', 'regional_optimization', 12.0, 3, '지역별 마케팅', 140.0
    )
    
    SELECT 
        -- 기준선 데이터 (모든 시나리오에 공통)
        baseline_metrics.*,
        
        -- 시나리오별 정보 (조인 키)
        scenarios.scenario_id,
        scenarios.scenario_name,
        scenarios.improvement_type,
        scenarios.improvement_pct,
        scenarios.priority_level,
        scenarios.action_item,
        scenarios.expected_roi,
        
        -- 시뮬레이션 결과 계산
        CASE 
            WHEN scenarios.improvement_type = 'retention' THEN 
                ROUND((baseline_repeat_rate + baseline_repeat_rate * scenarios.improvement_pct / 100) / baseline_repeat_rate * baseline_monthly_gmv - baseline_monthly_gmv, 2)
            WHEN scenarios.improvement_type = 'delivery' THEN 
                ROUND(baseline_monthly_gmv * 0.02, 2)  -- 배송 개선시 2% GMV 증가 가정
            WHEN scenarios.improvement_type = 'satisfaction' THEN 
                ROUND(baseline_monthly_gmv * 0.015, 2)  -- 만족도 개선시 1.5% GMV 증가 가정
            WHEN scenarios.improvement_type = 'shipping_cost' THEN 
                ROUND(baseline_monthly_gmv * baseline_shipping_ratio / 100 * scenarios.improvement_pct / 100, 2)  -- 배송비 절약액
            WHEN scenarios.improvement_type = 'vip_expansion' THEN 
                ROUND(vip_customers * avg_clv * scenarios.improvement_pct / 100, 2)  -- VIP 확대 효과
            WHEN scenarios.improvement_type = 'short_distance' THEN 
                ROUND(baseline_monthly_gmv * 0.008, 2)  -- 단거리 최적화 효과
            WHEN scenarios.improvement_type = 'new_customer_conversion' THEN 
                ROUND(baseline_monthly_gmv * 0.012, 2)  -- 신규 고객 전환 효과
            WHEN scenarios.improvement_type = 'regional_optimization' THEN 
                ROUND(baseline_monthly_gmv * 0.006, 2)  -- 지역별 최적화 효과
            ELSE 0
        END as expected_gmv_change,
        
        -- ROI 계산용 투자 비용 추정
        CASE 
            WHEN scenarios.improvement_type = 'retention' THEN ROUND(warm_customers * 50, 2)  -- 고객당 50 R$ 마케팅 비용
            WHEN scenarios.improvement_type = 'delivery' THEN ROUND(baseline_monthly_gmv * 0.01, 2)  -- GMV의 1% 물류 투자
            WHEN scenarios.improvement_type = 'satisfaction' THEN ROUND(baseline_monthly_gmv * 0.008, 2)  -- GMV의 0.8% 품질 투자
            WHEN scenarios.improvement_type = 'shipping_cost' THEN ROUND(baseline_monthly_gmv * 0.005, 2)  -- GMV의 0.5% 시스템 투자
            WHEN scenarios.improvement_type = 'vip_expansion' THEN ROUND(vip_customers * 100, 2)  -- VIP 고객당 100 R$ 서비스 비용
            WHEN scenarios.improvement_type = 'short_distance' THEN ROUND(baseline_monthly_gmv * 0.003, 2)  -- GMV의 0.3% 허브 투자
            WHEN scenarios.improvement_type = 'new_customer_conversion' THEN ROUND(baseline_monthly_gmv * 0.007, 2)  -- GMV의 0.7% 온보딩 투자
            WHEN scenarios.improvement_type = 'regional_optimization' THEN ROUND(baseline_monthly_gmv * 0.004, 2)  -- GMV의 0.4% 지역 투자
            ELSE 0
        END as estimated_investment,
        
        -- 영향받는 고객 수
        CASE 
            WHEN scenarios.improvement_type = 'retention' THEN warm_customers
            WHEN scenarios.improvement_type = 'delivery' THEN total_customers
            WHEN scenarios.improvement_type = 'satisfaction' THEN total_customers
            WHEN scenarios.improvement_type = 'shipping_cost' THEN total_customers
            WHEN scenarios.improvement_type = 'vip_expansion' THEN vip_customers
            WHEN scenarios.improvement_type = 'short_distance' THEN southeast_customers + south_customers  -- 주요 지역 고객
            WHEN scenarios.improvement_type = 'new_customer_conversion' THEN total_customers
            WHEN scenarios.improvement_type = 'regional_optimization' THEN northeast_customers  -- 개선 필요 지역
            ELSE 0
        END as affected_customers,
        
        -- 실행 난이도 (1: 쉬움, 5: 어려움)
        CASE 
            WHEN scenarios.improvement_type = 'retention' THEN 2
            WHEN scenarios.improvement_type = 'delivery' THEN 4
            WHEN scenarios.improvement_type = 'satisfaction' THEN 5
            WHEN scenarios.improvement_type = 'shipping_cost' THEN 3
            WHEN scenarios.improvement_type = 'vip_expansion' THEN 3
            WHEN scenarios.improvement_type = 'short_distance' THEN 3
            WHEN scenarios.improvement_type = 'new_customer_conversion' THEN 2
            WHEN scenarios.improvement_type = 'regional_optimization' THEN 4
            ELSE 3
        END as implementation_difficulty,
        
        -- 예상 실행 기간 (개월)
        CASE 
            WHEN scenarios.improvement_type = 'retention' THEN 2
            WHEN scenarios.improvement_type = 'delivery' THEN 6
            WHEN scenarios.improvement_type = 'satisfaction' THEN 12
            WHEN scenarios.improvement_type = 'shipping_cost' THEN 4
            WHEN scenarios.improvement_type = 'vip_expansion' THEN 6
            WHEN scenarios.improvement_type = 'short_distance' THEN 3
            WHEN scenarios.improvement_type = 'new_customer_conversion' THEN 4
            WHEN scenarios.improvement_type = 'regional_optimization' THEN 8
            ELSE 6
        END as implementation_months,
        
        -- 리스크 레벨 (1: 낮음, 5: 높음)
        CASE 
            WHEN scenarios.improvement_type = 'retention' THEN 2
            WHEN scenarios.improvement_type = 'delivery' THEN 3
            WHEN scenarios.improvement_type = 'satisfaction' THEN 4
            WHEN scenarios.improvement_type = 'shipping_cost' THEN 2
            WHEN scenarios.improvement_type = 'vip_expansion' THEN 3
            WHEN scenarios.improvement_type = 'short_distance' THEN 2
            WHEN scenarios.improvement_type = 'new_customer_conversion' THEN 3
            WHEN scenarios.improvement_type = 'regional_optimization' THEN 4
            ELSE 3
        END as risk_level,
        
        -- 계산용 ROI (연간 기준)
        ROUND(
            CASE 
                WHEN scenarios.improvement_type = 'retention' THEN 
                    (((baseline_repeat_rate + baseline_repeat_rate * scenarios.improvement_pct / 100) / baseline_repeat_rate * baseline_monthly_gmv - baseline_monthly_gmv) * 12) / NULLIF((warm_customers * 50), 0) * 100
                WHEN scenarios.improvement_type = 'delivery' THEN 
                    (baseline_monthly_gmv * 0.02 * 12) / NULLIF((baseline_monthly_gmv * 0.01), 0) * 100
                WHEN scenarios.improvement_type = 'satisfaction' THEN 
                    (baseline_monthly_gmv * 0.015 * 12) / NULLIF((baseline_monthly_gmv * 0.008), 0) * 100
                WHEN scenarios.improvement_type = 'shipping_cost' THEN 
                    (baseline_monthly_gmv * baseline_shipping_ratio / 100 * scenarios.improvement_pct / 100 * 12) / NULLIF((baseline_monthly_gmv * 0.005), 0) * 100
                WHEN scenarios.improvement_type = 'vip_expansion' THEN 
                    (vip_customers * avg_clv * scenarios.improvement_pct / 100) / NULLIF((vip_customers * 100), 0) * 100
                ELSE 150.0
            END, 1
        ) as calculated_roi_pct
        
    FROM baseline_metrics
    CROSS JOIN scenarios
    ORDER BY scenarios.priority_level, scenarios.expected_roi DESC
    """
    
    return save_datamart_csv(query, "dm_action_planner.csv", "액션 플래너")

In [22]:
# ====================================================================
# 7. 🔗 차원 테이블들 (태블로 조인용)
# ====================================================================

def create_dimension_tables():
    """태블로 조인용 차원 테이블들 생성 (DATE_FORMAT 수정)"""
    
    # 7-1. 고객 차원 테이블 (DATE_FORMAT 수정)
    customer_dim_query = """
    SELECT DISTINCT
        c.customer_unique_id,
        c.customer_id,
        c.customer_zip_code_prefix,
        c.customer_city,
        c.customer_state,
        
        -- 지역 그룹 (조인 키)
        CASE 
            WHEN c.customer_state IN ('SP', 'RJ', 'MG') THEN 'Southeast'
            WHEN c.customer_state IN ('RS', 'SC', 'PR') THEN 'South'
            WHEN c.customer_state IN ('BA', 'SE', 'AL', 'PE', 'PB', 'RN', 'CE', 'PI', 'MA') THEN 'Northeast'
            WHEN c.customer_state IN ('GO', 'DF', 'MT', 'MS') THEN 'Central-West'
            WHEN c.customer_state IN ('AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC') THEN 'North'
            ELSE 'Other'
        END as region_group,
        
        -- 지역 규모 분류
        CASE 
            WHEN c.customer_state IN ('SP', 'RJ') THEN 'Major'
            WHEN c.customer_state IN ('MG', 'RS', 'PR', 'SC', 'BA') THEN 'Large'
            ELSE 'Small/Medium'
        END as state_size,
        
        -- 경제 발전 수준
        CASE 
            WHEN c.customer_state IN ('SP', 'RJ', 'MG', 'RS', 'SC', 'PR') THEN 'Developed'
            WHEN c.customer_state IN ('GO', 'DF', 'MS', 'MT') THEN 'Developing'
            ELSE 'Emerging'
        END as economic_level,
        
        -- 첫 구매일 정보 (조인 키) - DATE_FORMAT 대신 CONCAT 사용
        first_purchase.first_purchase_date,
        CONCAT(YEAR(first_purchase.first_purchase_date), '-', 
               LPAD(MONTH(first_purchase.first_purchase_date), 2, '0'), '-01') as first_purchase_month,
        YEAR(first_purchase.first_purchase_date) as first_purchase_year,
        QUARTER(first_purchase.first_purchase_date) as first_purchase_quarter,
        
        -- 고객 요약 정보
        customer_summary.total_orders,
        customer_summary.total_gmv,
        customer_summary.lifecycle_days,
        
        -- 고객 세그먼트 (조인 키)
        CASE 
            WHEN customer_summary.total_orders = 1 THEN 'One-time'
            WHEN customer_summary.total_orders >= 3 AND customer_summary.total_gmv >= 500 THEN 'VIP'
            WHEN customer_summary.total_orders >= 2 THEN 'Repeat'
            ELSE 'Other'
        END as customer_segment,
        
        -- 위치 정보
        geo.avg_lat,
        geo.avg_lng
        
    FROM olist_customers c
    LEFT JOIN (
        SELECT 
            c.customer_unique_id,
            MIN(o.order_purchase_timestamp) as first_purchase_date
        FROM olist_customers c
        JOIN olist_orders o ON c.customer_id = o.customer_id
        WHERE o.order_status = 'delivered'
        GROUP BY c.customer_unique_id
    ) first_purchase ON c.customer_unique_id = first_purchase.customer_unique_id
    LEFT JOIN (
        SELECT 
            c.customer_unique_id,
            COUNT(DISTINCT o.order_id) as total_orders,
            ROUND(SUM(oi.price + oi.freight_value), 2) as total_gmv,
            DATEDIFF(MAX(o.order_purchase_timestamp), MIN(o.order_purchase_timestamp)) as lifecycle_days
        FROM olist_customers c
        JOIN olist_orders o ON c.customer_id = o.customer_id
        JOIN olist_order_items oi ON o.order_id = oi.order_id
        WHERE o.order_status = 'delivered'
        GROUP BY c.customer_unique_id
    ) customer_summary ON c.customer_unique_id = customer_summary.customer_unique_id
    LEFT JOIN (
        SELECT 
            geolocation_zip_code_prefix,
            AVG(geolocation_lat) as avg_lat,
            AVG(geolocation_lng) as avg_lng
        FROM olist_geolocation
        GROUP BY geolocation_zip_code_prefix
    ) geo ON c.customer_zip_code_prefix = geo.geolocation_zip_code_prefix
    """
    
    save_datamart_csv(customer_dim_query, "dim_customers.csv", "고객 차원 테이블")
    
    # 7-2. 시간 차원 테이블 (DATE_FORMAT 수정)
    time_dim_query = """
    SELECT DISTINCT
        CONCAT(YEAR(o.order_purchase_timestamp), '-', 
               LPAD(MONTH(o.order_purchase_timestamp), 2, '0'), '-01') as month_key,
        CONCAT(YEAR(o.order_purchase_timestamp), '-', 
               LPAD(MONTH(o.order_purchase_timestamp), 2, '0')) as month_name,
        YEAR(o.order_purchase_timestamp) as year,
        MONTH(o.order_purchase_timestamp) as month_num,
        QUARTER(o.order_purchase_timestamp) as quarter,
        
        -- 브라질 계절 (남반구)
        CASE 
            WHEN MONTH(o.order_purchase_timestamp) IN (12, 1, 2) THEN 'Summer'
            WHEN MONTH(o.order_purchase_timestamp) IN (3, 4, 5) THEN 'Autumn'
            WHEN MONTH(o.order_purchase_timestamp) IN (6, 7, 8) THEN 'Winter'
            ELSE 'Spring'
        END as season_br,
        
        -- 비즈니스 기간
        CASE 
            WHEN MONTH(o.order_purchase_timestamp) = 11 THEN 'Black Friday'
            WHEN MONTH(o.order_purchase_timestamp) = 12 THEN 'Christmas'
            WHEN MONTH(o.order_purchase_timestamp) IN (1, 2) THEN 'Summer Holiday'
            WHEN MONTH(o.order_purchase_timestamp) = 5 THEN 'Mothers Day'
            WHEN MONTH(o.order_purchase_timestamp) = 6 THEN 'Midyear'
            ELSE 'Regular'
        END as business_period,
        
        -- 요일 정보
        DAYOFWEEK(o.order_purchase_timestamp) as day_of_week,
        CASE 
            WHEN DAYOFWEEK(o.order_purchase_timestamp) IN (1, 7) THEN 'Weekend'
            ELSE 'Weekday'
        END as weekend_flag,
        
        -- 월 특성
        CASE 
            WHEN MONTH(o.order_purchase_timestamp) IN (11, 12) THEN 'High Season'
            WHEN MONTH(o.order_purchase_timestamp) IN (1, 2) THEN 'Post Holiday'
            WHEN MONTH(o.order_purchase_timestamp) IN (6, 7, 8) THEN 'Winter Low'
            ELSE 'Regular Season'
        END as sales_season
        
    FROM olist_orders o
    WHERE o.order_status = 'delivered'
    ORDER BY month_key
    """
    
    save_datamart_csv(time_dim_query, "dim_time.csv", "시간 차원 테이블")
    
    # 7-3. 지역 차원 테이블 (원래대로 유지 - 이미 성공)
    geography_dim_query = """
    SELECT DISTINCT
        c.customer_state as state_code,
        c.customer_city,
        c.customer_zip_code_prefix,
        
        -- 지역 그룹 (조인 키)
        CASE 
            WHEN c.customer_state IN ('SP', 'RJ', 'MG') THEN 'Southeast'
            WHEN c.customer_state IN ('RS', 'SC', 'PR') THEN 'South'
            WHEN c.customer_state IN ('BA', 'SE', 'AL', 'PE', 'PB', 'RN', 'CE', 'PI', 'MA') THEN 'Northeast'
            WHEN c.customer_state IN ('GO', 'DF', 'MT', 'MS') THEN 'Central-West'
            WHEN c.customer_state IN ('AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC') THEN 'North'
            ELSE 'Other'
        END as region_group,
        
        -- 주별 한국어 이름 (태블로 표시용)
        CASE 
            WHEN c.customer_state = 'SP' THEN '상파울루'
            WHEN c.customer_state = 'RJ' THEN '리우데자네이루'
            WHEN c.customer_state = 'MG' THEN '미나스제라이스'
            WHEN c.customer_state = 'RS' THEN '리우그란데도술'
            WHEN c.customer_state = 'PR' THEN '파라나'
            WHEN c.customer_state = 'SC' THEN '산타카타리나'
            WHEN c.customer_state = 'BA' THEN '바이아'
            WHEN c.customer_state = 'GO' THEN '고이아스'
            WHEN c.customer_state = 'DF' THEN '연방구'
            WHEN c.customer_state = 'PE' THEN '페르남부쿠'
            WHEN c.customer_state = 'CE' THEN '세아라'
            WHEN c.customer_state = 'MA' THEN '마라냥'
            ELSE c.customer_state
        END as state_name_kr,
        
        -- 경제 규모 분류 (조인 키)
        CASE 
            WHEN c.customer_state IN ('SP', 'RJ') THEN 'Tier 1'
            WHEN c.customer_state IN ('MG', 'RS', 'PR', 'SC', 'BA') THEN 'Tier 2'
            ELSE 'Tier 3'
        END as economic_tier,
        
        -- 지역 특성
        CASE 
            WHEN c.customer_state IN ('SP', 'RJ') THEN 'Metropolitan'
            WHEN c.customer_state IN ('MG', 'RS', 'PR', 'SC') THEN 'Industrial'
            WHEN c.customer_state IN ('BA', 'PE', 'CE') THEN 'Tourism'
            ELSE 'Developing'
        END as region_characteristic,
        
        -- 위치 정보 (평균 위도/경도)
        geo.avg_lat,
        geo.avg_lng,
        geo.location_count,
        
        -- 비즈니스 성과 기준
        regional_performance.avg_order_value,
        regional_performance.avg_rating,
        regional_performance.delivery_delay_rate
        
    FROM olist_customers c
    LEFT JOIN (
        SELECT 
            geolocation_zip_code_prefix,
            AVG(geolocation_lat) as avg_lat,
            AVG(geolocation_lng) as avg_lng,
            COUNT(*) as location_count
        FROM olist_geolocation
        GROUP BY geolocation_zip_code_prefix
    ) geo ON c.customer_zip_code_prefix = geo.geolocation_zip_code_prefix
    LEFT JOIN (
        SELECT 
            cust.customer_state,
            ROUND(AVG(oi.price + oi.freight_value), 2) as avg_order_value,
            ROUND(AVG(r.review_score), 2) as avg_rating,
            ROUND(AVG(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 ELSE 0 END) * 100, 1) as delivery_delay_rate
        FROM olist_customers cust
        JOIN olist_orders o ON cust.customer_id = o.customer_id
        JOIN olist_order_items oi ON o.order_id = oi.order_id
        LEFT JOIN olist_order_reviews r ON o.order_id = r.order_id
        WHERE o.order_status = 'delivered'
        GROUP BY cust.customer_state
    ) regional_performance ON c.customer_state = regional_performance.customer_state
    """
    
    save_datamart_csv(geography_dim_query, "dim_geography.csv", "지역 차원 테이블")
    
    # 7-4. 상품 카테고리 차원 테이블 (원래대로 유지 - 이미 성공)
    category_dim_query = """
    SELECT DISTINCT
        p.product_category_name as category_portuguese,
        pt.product_category_name_english as category_english,
        
        -- 대분류 그룹 (조인 키)
        CASE 
            WHEN pt.product_category_name_english IN ('bed_bath_table', 'furniture_decor', 'housewares', 'home_decoration', 'furniture_bedroom', 'furniture_living_room', 'kitchen_dining_laundry_garden_furniture', 'garden_tools', 'home_construction') 
                THEN 'Home & Garden'
            WHEN pt.product_category_name_english IN ('computers_accessories', 'electronics', 'telephony', 'watches_gifts', 'auto', 'computers', 'tablets_printing_image')
                THEN 'Electronics & Tech'
            WHEN pt.product_category_name_english IN ('fashion_bags_accessories', 'fashion_female_clothing', 'fashion_male_clothing', 'fashion_shoes', 'fashion_underwear_beach', 'perfumery', 'health_beauty')
                THEN 'Fashion & Beauty'
            WHEN pt.product_category_name_english IN ('sports_leisure', 'toys', 'cool_stuff', 'party_supplies', 'christmas_supplies')
                THEN 'Sports & Leisure'
            WHEN pt.product_category_name_english IN ('office_furniture', 'stationery', 'office_supplies')
                THEN 'Office & Supplies'
            WHEN pt.product_category_name_english IN ('baby', 'diapers_and_hygiene', 'fashion_childrens_clothes')
                THEN 'Baby & Kids'
            WHEN pt.product_category_name_english IN ('books_general_interest', 'books_technical', 'books_imported', 'cds_dvds_musicals', 'music', 'dvds_blu_ray')
                THEN 'Books & Media'
            WHEN pt.product_category_name_english IN ('food_drink', 'pet_shop')
                THEN 'Food & Pets'
            WHEN pt.product_category_name_english IN ('art', 'arts_and_craftmanship', 'construction_tools_construction', 'construction_tools_lights', 'construction_tools_tools', 'construction_tools_safety', 'industry_commerce_and_business')
                THEN 'Art & Construction'
            ELSE 'Other'
        END as category_group,
        
        -- 카테고리 특성 분류
        CASE 
            WHEN pt.product_category_name_english IN ('electronics', 'computers', 'telephony', 'watches_gifts') THEN 'High Tech'
            WHEN pt.product_category_name_english IN ('fashion_female_clothing', 'fashion_male_clothing', 'fashion_shoes', 'perfumery') THEN 'Fashion'
            WHEN pt.product_category_name_english IN ('bed_bath_table', 'furniture_decor', 'housewares') THEN 'Home Essentials'
            WHEN pt.product_category_name_english IN ('toys', 'sports_leisure', 'cool_stuff') THEN 'Lifestyle'
            WHEN pt.product_category_name_english IN ('baby', 'health_beauty', 'diapers_and_hygiene') THEN 'Personal Care'
            ELSE 'General'
        END as category_type,
        
        -- 가격대 분류 (카테고리별 평균 가격 기준)
        CASE 
            WHEN category_stats.avg_price >= 200 THEN 'Premium'
            WHEN category_stats.avg_price >= 100 THEN 'Mid-Range'
            WHEN category_stats.avg_price >= 50 THEN 'Standard'
            ELSE 'Budget'
        END as price_tier,
        
        -- 배송 복잡도
        CASE 
            WHEN pt.product_category_name_english IN ('furniture_decor', 'furniture_bedroom', 'furniture_living_room', 'home_construction') THEN 'Complex'
            WHEN pt.product_category_name_english IN ('electronics', 'computers', 'tablets_printing_image') THEN 'Standard'
            WHEN pt.product_category_name_english IN ('books_general_interest', 'stationery', 'health_beauty') THEN 'Simple'
            ELSE 'Standard'
        END as shipping_complexity,
        
        -- 카테고리 성과 지표
        category_stats.total_orders,
        category_stats.total_revenue,
        category_stats.avg_price,
        category_stats.avg_rating
        
    FROM olist_products p
    LEFT JOIN product_category_name_translation pt ON p.product_category_name = pt.product_category_name
    LEFT JOIN (
        SELECT 
            p.product_category_name,
            COUNT(DISTINCT oi.order_id) as total_orders,
            ROUND(SUM(oi.price), 2) as total_revenue,
            ROUND(AVG(oi.price), 2) as avg_price,
            ROUND(AVG(r.review_score), 2) as avg_rating
        FROM olist_products p
        JOIN olist_order_items oi ON p.product_id = oi.product_id
        JOIN olist_orders o ON oi.order_id = o.order_id
        LEFT JOIN olist_order_reviews r ON o.order_id = r.order_id
        WHERE o.order_status = 'delivered'
        GROUP BY p.product_category_name
    ) category_stats ON p.product_category_name = category_stats.product_category_name
    WHERE p.product_category_name IS NOT NULL
    """
    
    save_datamart_csv(category_dim_query, "dim_categories.csv", "카테고리 차원 테이블")
    
    print("\n🎉 모든 차원 테이블 생성 완료!")
    return True

In [26]:
def create_aggregated_metrics():
    """태블로 성능 최적화용 사전 집계 테이블들 (DATE_FORMAT 수정)"""
    
    # 8-1. 월별 KPI 집계 (빠른 트렌드 차트용) - GROUP BY 수정
    monthly_kpi_query = """
    WITH monthly_base AS (
        SELECT 
            YEAR(o.order_purchase_timestamp) as year,
            MONTH(o.order_purchase_timestamp) as month_num,
            QUARTER(o.order_purchase_timestamp) as quarter,
            
            -- 기본 지표
            COUNT(DISTINCT o.order_id) as total_orders,
            COUNT(DISTINCT c.customer_unique_id) as unique_customers,
            ROUND(SUM(oi.price + oi.freight_value), 2) as gmv,
            ROUND(SUM(oi.price), 2) as product_revenue,
            ROUND(SUM(oi.freight_value), 2) as shipping_revenue,
            
            -- 고객 유형별 (단순화)
            COUNT(DISTINCT CASE WHEN first_purchase.is_first_month = 1 THEN c.customer_unique_id END) as new_customers,
            COUNT(DISTINCT CASE WHEN first_purchase.is_first_month = 0 THEN c.customer_unique_id END) as returning_customers,
            
            -- 만족도 지표
            ROUND(AVG(r.review_score), 2) as avg_rating,
            ROUND(COUNT(CASE WHEN r.review_score >= 4 THEN 1 END) * 100.0 / NULLIF(COUNT(r.review_score), 0), 1) as positive_review_pct,
            ROUND(COUNT(CASE WHEN r.review_score <= 2 THEN 1 END) * 100.0 / NULLIF(COUNT(r.review_score), 0), 1) as negative_review_pct,
            
            -- 배송 지표
            ROUND(AVG(DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp)), 1) as avg_delivery_days,
            ROUND(COUNT(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 END) * 100.0 / COUNT(*), 1) as delay_rate_pct
            
        FROM olist_orders o
        JOIN olist_customers c ON o.customer_id = c.customer_id
        JOIN olist_order_items oi ON o.order_id = oi.order_id
        LEFT JOIN olist_order_reviews r ON o.order_id = r.order_id
        LEFT JOIN (
            SELECT 
                c.customer_unique_id,
                CONCAT(YEAR(o.order_purchase_timestamp), '-', 
                       LPAD(MONTH(o.order_purchase_timestamp), 2, '0'), '-01') as order_month,
                CASE WHEN CONCAT(YEAR(o.order_purchase_timestamp), '-', 
                                LPAD(MONTH(o.order_purchase_timestamp), 2, '0'), '-01') = 
                          CONCAT(YEAR(MIN(o.order_purchase_timestamp) OVER (PARTITION BY c.customer_unique_id)), '-', 
                                 LPAD(MONTH(MIN(o.order_purchase_timestamp) OVER (PARTITION BY c.customer_unique_id)), 2, '0'), '-01') 
                     THEN 1 ELSE 0 END as is_first_month
            FROM olist_customers c
            JOIN olist_orders o ON c.customer_id = o.customer_id
            WHERE o.order_status = 'delivered'
        ) first_purchase ON c.customer_unique_id = first_purchase.customer_unique_id 
                       AND CONCAT(YEAR(o.order_purchase_timestamp), '-', 
                                 LPAD(MONTH(o.order_purchase_timestamp), 2, '0'), '-01') = first_purchase.order_month
        
        WHERE o.order_status = 'delivered'
          AND o.order_delivered_customer_date IS NOT NULL
        GROUP BY 
            YEAR(o.order_purchase_timestamp),
            MONTH(o.order_purchase_timestamp),
            QUARTER(o.order_purchase_timestamp)
    )
    
    SELECT 
        CONCAT(year, '-', LPAD(month_num, 2, '0'), '-01') as month_key,
        year,
        month_num,
        quarter,
        
        -- 기본 지표
        total_orders,
        unique_customers,
        gmv,
        product_revenue,
        shipping_revenue,
        new_customers,
        returning_customers,
        
        -- 만족도 지표
        avg_rating,
        positive_review_pct,
        negative_review_pct,
        
        -- 배송 지표
        avg_delivery_days,
        delay_rate_pct,
        
        -- 효율성 지표
        ROUND(gmv / total_orders, 2) as aov,
        ROUND(shipping_revenue * 100.0 / gmv, 1) as shipping_ratio_pct,
        
        -- 성장률 계산용
        LAG(total_orders) OVER (ORDER BY year, month_num) as prev_orders,
        LAG(gmv) OVER (ORDER BY year, month_num) as prev_gmv,
        LAG(unique_customers) OVER (ORDER BY year, month_num) as prev_customers
        
    FROM monthly_base
    ORDER BY year, month_num
    """
    
    save_datamart_csv(monthly_kpi_query, "agg_monthly_kpi.csv", "월별 KPI 집계")
    
    # 8-2. 지역별 성과 집계 (지도 시각화용) - 원래대로 유지 (DATE_FORMAT 없음)
    regional_performance_query = """
    SELECT 
        c.customer_state,
        
        -- 지역 정보 (조인 키)
        CASE 
            WHEN c.customer_state IN ('SP', 'RJ', 'MG') THEN 'Southeast'
            WHEN c.customer_state IN ('RS', 'SC', 'PR') THEN 'South'
            WHEN c.customer_state IN ('BA', 'SE', 'AL', 'PE', 'PB', 'RN', 'CE', 'PI', 'MA') THEN 'Northeast'
            WHEN c.customer_state IN ('GO', 'DF', 'MT', 'MS') THEN 'Central-West'
            WHEN c.customer_state IN ('AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC') THEN 'North'
            ELSE 'Other'
        END as region_group,
        
        -- 기본 지표
        COUNT(DISTINCT o.order_id) as total_orders,
        COUNT(DISTINCT c.customer_unique_id) as unique_customers,
        ROUND(SUM(oi.price + oi.freight_value), 2) as total_gmv,
        
        -- 평균 지표
        ROUND(AVG(oi.price + oi.freight_value), 2) as avg_order_value,
        ROUND(SUM(oi.price + oi.freight_value) / COUNT(DISTINCT c.customer_unique_id), 2) as gmv_per_customer,
        ROUND(COUNT(DISTINCT o.order_id) / COUNT(DISTINCT c.customer_unique_id), 2) as orders_per_customer,
        
        -- 만족도
        ROUND(AVG(r.review_score), 2) as avg_rating,
        ROUND(COUNT(CASE WHEN r.review_score >= 4 THEN 1 END) * 100.0 / NULLIF(COUNT(r.review_score), 0), 1) as positive_review_pct,
        ROUND(COUNT(CASE WHEN r.review_score <= 2 THEN 1 END) * 100.0 / NULLIF(COUNT(r.review_score), 0), 1) as negative_review_pct,
        
        -- 배송 성과
        ROUND(AVG(DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp)), 1) as avg_delivery_days,
        ROUND(COUNT(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 END) * 100.0 / COUNT(*), 1) as delay_rate_pct,
        
        -- 배송비
        ROUND(AVG(oi.freight_value), 2) as avg_shipping_cost,
        ROUND(SUM(oi.freight_value) * 100.0 / SUM(oi.price + oi.freight_value), 1) as shipping_ratio_pct,
        
        -- 시장 점유율
        ROUND(SUM(oi.price + oi.freight_value) * 100.0 / SUM(SUM(oi.price + oi.freight_value)) OVER(), 2) as market_share_pct,
        
        -- 성과 순위 (태블로 필터용)
        RANK() OVER (ORDER BY SUM(oi.price + oi.freight_value) DESC) as gmv_rank,
        RANK() OVER (ORDER BY AVG(r.review_score) DESC) as satisfaction_rank,
        RANK() OVER (ORDER BY AVG(DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp))) as delivery_rank,
        
        -- 고객 세그먼트 분포
        COUNT(DISTINCT CASE WHEN customer_segments.segment = 'VIP' THEN c.customer_unique_id END) as vip_customers,
        COUNT(DISTINCT CASE WHEN customer_segments.segment = 'Repeat' THEN c.customer_unique_id END) as repeat_customers,
        COUNT(DISTINCT CASE WHEN customer_segments.segment = 'One-time' THEN c.customer_unique_id END) as onetime_customers
        
    FROM olist_orders o
    JOIN olist_customers c ON o.customer_id = c.customer_id
    JOIN olist_order_items oi ON o.order_id = oi.order_id
    LEFT JOIN olist_order_reviews r ON o.order_id = r.order_id
    LEFT JOIN (
        SELECT 
            c.customer_unique_id,
            CASE 
                WHEN COUNT(DISTINCT o.order_id) >= 3 AND SUM(oi.price + oi.freight_value) >= 500 THEN 'VIP'
                WHEN COUNT(DISTINCT o.order_id) >= 2 THEN 'Repeat'
                ELSE 'One-time'
            END as segment
        FROM olist_customers c
        JOIN olist_orders o ON c.customer_id = o.customer_id
        JOIN olist_order_items oi ON o.order_id = oi.order_id
        WHERE o.order_status = 'delivered'
        GROUP BY c.customer_unique_id
    ) customer_segments ON c.customer_unique_id = customer_segments.customer_unique_id
    
    WHERE o.order_status = 'delivered'
      AND o.order_delivered_customer_date IS NOT NULL
    GROUP BY c.customer_state
    HAVING total_orders >= 10  -- 최소 주문량 필터
    ORDER BY total_gmv DESC
    """
    
    save_datamart_csv(regional_performance_query, "agg_regional_performance.csv", "지역별 성과 집계")
    
    # 8-3. 카테고리별 성과 집계 (원래대로 유지 - DATE_FORMAT 없음)
    category_performance_query = """
    SELECT 
        pt.product_category_name_english as category_english,
        p.product_category_name as category_portuguese,
        
        -- 대분류 그룹 (조인 키)
        CASE 
            WHEN pt.product_category_name_english IN ('bed_bath_table', 'furniture_decor', 'housewares', 'home_decoration', 'furniture_bedroom', 'furniture_living_room') 
                THEN 'Home & Garden'
            WHEN pt.product_category_name_english IN ('computers_accessories', 'electronics', 'telephony', 'watches_gifts', 'auto') 
                THEN 'Electronics & Tech'
            WHEN pt.product_category_name_english IN ('fashion_bags_accessories', 'fashion_female_clothing', 'fashion_male_clothing', 'fashion_shoes', 'perfumery', 'health_beauty') 
                THEN 'Fashion & Beauty'
            WHEN pt.product_category_name_english IN ('sports_leisure', 'toys', 'cool_stuff', 'party_supplies') 
                THEN 'Sports & Leisure'
            WHEN pt.product_category_name_english IN ('baby', 'diapers_and_hygiene', 'fashion_childrens_clothes') 
                THEN 'Baby & Kids'
            WHEN pt.product_category_name_english IN ('books_general_interest', 'books_technical', 'cds_dvds_musicals', 'music') 
                THEN 'Books & Media'
            ELSE 'Other'
        END as category_group,
        
        -- 기본 성과 지표
        COUNT(DISTINCT oi.order_id) as total_orders,
        COUNT(DISTINCT oi.product_id) as unique_products,
        COUNT(DISTINCT oi.seller_id) as unique_sellers,
        ROUND(SUM(oi.price), 2) as total_revenue,
        ROUND(SUM(oi.freight_value), 2) as total_shipping,
        
        -- 평균 지표
        ROUND(AVG(oi.price), 2) as avg_price,
        ROUND(AVG(oi.freight_value), 2) as avg_shipping_cost,
        ROUND(SUM(oi.freight_value) / SUM(oi.price + oi.freight_value) * 100, 1) as shipping_ratio_pct,
        
        -- 만족도
        ROUND(AVG(r.review_score), 2) as avg_rating,
        ROUND(COUNT(CASE WHEN r.review_score >= 4 THEN 1 END) * 100.0 / NULLIF(COUNT(r.review_score), 0), 1) as positive_review_pct,
        
        -- 시장 점유율
        ROUND(SUM(oi.price) * 100.0 / SUM(SUM(oi.price)) OVER(), 2) as market_share_pct,
        
        -- 성과 순위
        RANK() OVER (ORDER BY SUM(oi.price) DESC) as revenue_rank,
        RANK() OVER (ORDER BY AVG(r.review_score) DESC) as satisfaction_rank,
        RANK() OVER (ORDER BY COUNT(DISTINCT oi.order_id) DESC) as popularity_rank
        
    FROM olist_order_items oi
    JOIN olist_products p ON oi.product_id = p.product_id
    JOIN olist_orders o ON oi.order_id = o.order_id
    LEFT JOIN product_category_name_translation pt ON p.product_category_name = pt.product_category_name
    LEFT JOIN olist_order_reviews r ON o.order_id = r.order_id
    WHERE o.order_status = 'delivered'
      AND pt.product_category_name_english IS NOT NULL
    GROUP BY pt.product_category_name_english, p.product_category_name
    HAVING total_orders >= 50  -- 최소 주문량 필터
    ORDER BY total_revenue DESC
    """
    
    save_datamart_csv(category_performance_query, "agg_category_performance.csv", "카테고리별 성과 집계")
    
    # 8-4. 고객 세그먼트별 집계 (원래대로 유지 - DATE_FORMAT 없음)
    segment_performance_query = """
    SELECT 
        customer_data.rfm_segment,
        customer_data.region_group,
        
        -- 기본 지표
        COUNT(*) as customer_count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as customer_pct,
        
        -- 매출 기여도
        ROUND(SUM(customer_data.monetary), 2) as total_gmv,
        ROUND(SUM(customer_data.monetary) * 100.0 / SUM(SUM(customer_data.monetary)) OVER(), 2) as gmv_pct,
        
        -- 평균 지표
        ROUND(AVG(customer_data.monetary), 2) as avg_clv,
        ROUND(AVG(customer_data.frequency), 2) as avg_orders,
        ROUND(AVG(customer_data.avg_order_value), 2) as avg_order_value,
        ROUND(AVG(customer_data.recency_days), 1) as avg_recency_days,
        
        -- 만족도
        ROUND(AVG(customer_data.avg_rating), 2) as avg_rating,
        ROUND(AVG(customer_data.positive_review_pct), 1) as avg_positive_pct,
        
        -- 배송 성과
        ROUND(AVG(customer_data.avg_delivery_days), 1) as avg_delivery_days,
        ROUND(AVG(customer_data.delay_rate_pct), 1) as avg_delay_rate,
        
        -- 다양성 지표
        ROUND(AVG(customer_data.category_diversity), 1) as avg_category_diversity,
        ROUND(AVG(customer_data.seller_diversity), 1) as avg_seller_diversity
        
    FROM (
        SELECT 
            c.customer_unique_id,
            
            -- 세그먼트 분류
            CASE 
                WHEN COUNT(DISTINCT o.order_id) >= 3 AND SUM(oi.price + oi.freight_value) >= 500 THEN 'VIP'
                WHEN COUNT(DISTINCT o.order_id) >= 2 AND SUM(oi.price + oi.freight_value) >= 300 THEN 'Loyal'
                WHEN DATEDIFF('2018-10-01', MAX(o.order_purchase_timestamp)) <= 180 THEN 'Warm'
                WHEN DATEDIFF('2018-10-01', MAX(o.order_purchase_timestamp)) <= 365 THEN 'Cold'
                ELSE 'At-Risk'
            END as rfm_segment,
            
            -- 지역 그룹
            CASE 
                WHEN c.customer_state IN ('SP', 'RJ', 'MG') THEN 'Southeast'
                WHEN c.customer_state IN ('RS', 'SC', 'PR') THEN 'South'
                WHEN c.customer_state IN ('BA', 'SE', 'AL', 'PE', 'PB', 'RN', 'CE', 'PI', 'MA') THEN 'Northeast'
                WHEN c.customer_state IN ('GO', 'DF', 'MT', 'MS') THEN 'Central-West'
                WHEN c.customer_state IN ('AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC') THEN 'North'
                ELSE 'Other'
            END as region_group,
            
            -- RFM 지표
            DATEDIFF('2018-10-01', MAX(o.order_purchase_timestamp)) as recency_days,
            COUNT(DISTINCT o.order_id) as frequency,
            ROUND(SUM(oi.price + oi.freight_value), 2) as monetary,
            ROUND(AVG(oi.price + oi.freight_value), 2) as avg_order_value,
            
            -- 만족도
            ROUND(AVG(r.review_score), 2) as avg_rating,
            ROUND(AVG(CASE WHEN r.review_score >= 4 THEN 100 ELSE 0 END), 1) as positive_review_pct,
            
            -- 배송 성과
            ROUND(AVG(DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp)), 1) as avg_delivery_days,
            ROUND(AVG(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 100 ELSE 0 END), 1) as delay_rate_pct,
            
            -- 다양성
            COUNT(DISTINCT p.product_category_name) as category_diversity,
            COUNT(DISTINCT oi.seller_id) as seller_diversity
            
        FROM olist_customers c
        JOIN olist_orders o ON c.customer_id = o.customer_id
        JOIN olist_order_items oi ON o.order_id = oi.order_id
        JOIN olist_products p ON oi.product_id = p.product_id
        LEFT JOIN olist_order_reviews r ON o.order_id = r.order_id
        WHERE o.order_status = 'delivered'
        GROUP BY c.customer_unique_id, c.customer_state
    ) customer_data
    GROUP BY customer_data.rfm_segment, customer_data.region_group
    ORDER BY total_gmv DESC
    """
    
    save_datamart_csv(segment_performance_query, "agg_segment_performance.csv", "세그먼트별 성과 집계")
    
    print("\n🎉 모든 집계 테이블 생성 완료!")
    return True

In [11]:
def create_all_datamarts():
    """모든 데이터마트 생성 (전체 실행)"""
    print("🚀 Olist 전체 데이터마트 생성을 시작합니다!")
    print("="*70)
    
    datamarts = [
        ("1. 프로젝트 개요", create_project_overview_mart),
        ("2. KPI 개요", create_kpi_overview_mart),
        ("3. 만족도 분석", create_satisfaction_analysis_mart),
        ("4. 리뷰·배송 성과", create_review_shipping_mart),
        ("5. 고객 세그먼트", create_customer_segment_mart),
        ("6. 액션 플래너", create_action_planner_mart),
        ("7. 차원 테이블들", create_dimension_tables),
        ("8. 집계 테이블들", create_aggregated_metrics),
    ]
    
    success_count = 0
    total_start_time = time.time()
    
    for description, func in datamarts:
        print(f"\n{'='*50}")
        print(f"📊 {description} 데이터마트 생성 중...")
        print(f"{'='*50}")
        
        try:
            if func():
                success_count += 1
            else:
                print(f"❌ {description} 생성 실패!")
        except Exception as e:
            print(f"❌ {description} 생성 중 오류 발생: {e}")
    
    # 결과 요약
    total_elapsed = time.time() - total_start_time
    print(f"\n🎉 데이터마트 생성 완료!")
    print(f"{'='*70}")
    print(f"✅ 성공: {success_count}/{len(datamarts)}개")
    print(f"⏱️  총 소요시간: {total_elapsed:.1f}초")
    print(f"💾 저장 경로: {os.path.abspath(EXPORT_DIR)}")
    
    # 생성된 파일 목록
    if success_count == len(datamarts):
        print(f"\n📂 생성된 파일 목록:")
        print(f"   📊 분석용 데이터마트 (6개):")
        print(f"      - dm_project_overview.csv")
        print(f"      - dm_kpi_overview.csv")
        print(f"      - dm_satisfaction_analysis.csv")
        print(f"      - dm_review_shipping.csv")
        print(f"      - dm_customer_segment.csv")
        print(f"      - dm_action_planner.csv")
        print(f"   🔗 차원 테이블 (4개):")
        print(f"      - dim_customers.csv")
        print(f"      - dim_time.csv")
        print(f"      - dim_geography.csv")
        print(f"      - dim_categories.csv")
        print(f"   📈 집계 테이블 (4개):")
        print(f"      - agg_monthly_kpi.csv")
        print(f"      - agg_regional_performance.csv")
        print(f"      - agg_category_performance.csv")
        print(f"      - agg_segment_performance.csv")
        print(f"\n🎯 총 14개 파일이 성공적으로 생성되었습니다!")

def create_core_datamarts():
    """핵심 데이터마트만 생성 (빠른 테스트용)"""
    print("🔥 핵심 데이터마트만 생성합니다!")
    
    core_datamarts = [
        ("KPI 개요", create_kpi_overview_mart),
        ("만족도 분석", create_satisfaction_analysis_mart),
        ("고객 세그먼트", create_customer_segment_mart),
        ("월별 KPI 집계", lambda: create_aggregated_metrics() and print("월별 KPI만 생성됨")),
    ]
    
    for description, func in core_datamarts:
        print(f"\n🔄 {description} 생성 중...")
        func()

def create_single_datamart(datamart_name):
    """개별 데이터마트 생성"""
    datamart_map = {
        "project": create_project_overview_mart,
        "kpi": create_kpi_overview_mart,
        "satisfaction": create_satisfaction_analysis_mart,
        "shipping": create_review_shipping_mart,
        "segment": create_customer_segment_mart,
        "action": create_action_planner_mart,
        "dimensions": create_dimension_tables,
        "aggregations": create_aggregated_metrics,
    }
    
    if datamart_name in datamart_map:
        print(f"🎯 {datamart_name} 데이터마트만 생성합니다!")
        datamart_map[datamart_name]()
    else:
        print(f"❌ '{datamart_name}' 데이터마트를 찾을 수 없습니다.")
        print(f"사용 가능한 옵션: {list(datamart_map.keys())}")

In [27]:
create_aggregated_metrics()


🔄 월별 KPI 집계 데이터마트 생성 중...
✅ agg_monthly_kpi.csv 저장 완료!
   📊 23 rows × 21 columns
   ⏱️  소요시간: 4.3초
   💾 파일경로: ../data\agg_monthly_kpi.csv

🔄 지역별 성과 집계 데이터마트 생성 중...
✅ agg_regional_performance.csv 저장 완료!
   📊 27 rows × 22 columns
   ⏱️  소요시간: 4.1초
   💾 파일경로: ../data\agg_regional_performance.csv

🔄 카테고리별 성과 집계 데이터마트 생성 중...
✅ agg_category_performance.csv 저장 완료!
   📊 58 rows × 17 columns
   ⏱️  소요시간: 13.4초
   💾 파일경로: ../data\agg_category_performance.csv

🔄 세그먼트별 성과 집계 데이터마트 생성 중...
✅ agg_segment_performance.csv 저장 완료!
   📊 30 rows × 16 columns
   ⏱️  소요시간: 3.3초
   💾 파일경로: ../data\agg_segment_performance.csv

🎉 모든 집계 테이블 생성 완료!


True

In [43]:
create_all_datamarts()

🚀 Olist 전체 데이터마트 생성을 시작합니다!

📊 1. 프로젝트 개요 데이터마트 생성 중...

🔄 프로젝트 개요 데이터마트 생성 중...
✅ dm_project_overview.csv 저장 완료!
   📊 3 rows × 24 columns
   ⏱️  소요시간: 3.3초
   💾 파일경로: ../data\dm_project_overview.csv

📊 2. KPI 개요 데이터마트 생성 중...

🔄 KPI 개요 데이터마트 생성 중...
❌ dm_kpi_overview.csv 생성 실패: unsupported format character '?' (0xc774) at index 36
❌ 2. KPI 개요 생성 실패!

📊 3. 만족도 분석 데이터마트 생성 중...

🔄 만족도 분석 데이터마트 생성 중...
❌ dm_satisfaction_analysis.csv 생성 실패: unsupported format character 'Y' (0x59) at index 82
❌ 3. 만족도 분석 생성 실패!

📊 4. 리뷰·배송 성과 데이터마트 생성 중...

🔄 리뷰·배송 성과 데이터마트 생성 중...
❌ dm_review_shipping.csv 생성 실패: unsupported format character 'Y' (0x59) at index 82
❌ 4. 리뷰·배송 성과 생성 실패!

📊 5. 고객 세그먼트 데이터마트 생성 중...

🔄 고객 세그먼트 & 리텐션 데이터마트 생성 중...
❌ dm_customer_segment.csv 생성 실패: unsupported format character 'Y' (0x59) at index 1835
❌ 5. 고객 세그먼트 생성 실패!

📊 6. 액션 플래너 데이터마트 생성 중...

🔄 액션 플래너 데이터마트 생성 중...
❌ dm_action_planner.csv 생성 실패: unsupported format character 'Y' (0x59) at index 2609
❌ 6. 액션 플래너 생성 실패!

📊 7. 